In [ ]:
# ----------------------------------------------------------------------
#  Random Search (랜덤 탐색) 정의 및 원리
# ----------------------------------------------------------------------

# 1. 정의:
# - 사용자가 정의한 하이퍼파라미터의 탐색 공간(범위 또는 분포) 내에서 '정해진 횟수'만큼 '무작위'로 조합을 샘플링하여 성능을 평가하는 방식.
# - Grid Search의 대안으로, 탐색 시간 대비 효율을 높이는 데 목적이 있음.

# 2. 핵심 원리:
# - 탐색 공간 정의: 각 하이퍼파라미터의 후보 값 리스트나 확률 분포(예: 균일 분포, 정규 분포)를 정의.
# - 무작위 샘플링: 사용자가 지정한 횟수(n_iter)만큼 조합을 무작위로 추출하여 시도.
# - 교차 검증 (CV): 추출된 각 조합에 대해 K-Fold 교차 검증을 수행하여 일반화 성능(평균 점수)을 평가.
# - 최적 선택: 시도 횟수 내에서 가장 높은 평균 점수를 기록한 조합을 선택.

# 3. Grid Search 대비 장점:
# - 효율성: 실제로 성능에 큰 영향을 미치는 파라미터 값을 탐색할 확률이 높음. (Grid Search는 중요하지 않은 파라미터 축을 불필요하게 탐색함).
# - 속도: 탐색 횟수(n_iter)를 직접 제어하므로, Grid Search보다 훨씬 빠르게 튜닝을 완료할 수 있음.

# 4. 단점:
# - 불확실성: Grid Search와 달리 정의된 탐색 공간 전체를 확인하지 않기 때문에, 운이 나쁘면 최적의 조합을 놓칠 수 있음.
# - 탐색 횟수 의존성: n_iter 설정이 너무 작으면 성능이 떨어질 수 있음.

# ----------------------------------------------------------------------
#  Python에서의 구현 (Scikit-learn)
# ----------------------------------------------------------------------

# - 클래스: sklearn.model_selection.RandomizedSearchCV
# - 주요 매개변수:
#   - param_distributions: 하이퍼파라미터의 확률 분포 또는 후보 리스트를 담은 딕셔너리.
#   - n_iter: 무작위로 샘플링할 횟수 (탐색 시간 제어).
#   - cv: 사용할 교차 검증 폴드 개수.
#   - scoring: 평가 지표.

In [ ]:
# --------------------
#  라이브러리 및 데이터 준비
# --------------------
import numpy as np
from sklearn.model_selection import RandomizedSearchCV # Random Search 함수
from sklearn.ensemble import RandomForestClassifier # 탐색 대상 모델
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from scipy.stats import randint, uniform # 랜덤 샘플링을 위한 분포 함수

# Iris 데이터셋 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("--- 데이터 준비 완료 ---")

In [ ]:
# --------------------
#  2단계: 탐색 공간 (Parameter Distribution) 정의
# --------------------
# n_estimators: 100부터 500까지의 정수 범위에서 균일하게 샘플링
# max_depth: 3부터 10까지의 정수 범위에서 균일하게 샘플링
# min_samples_split: 2부터 10까지의 정수 범위에서 균일하게 샘플링
param_distributions = {
    'n_estimators': randint(low=100, high=500),
    'max_depth': randint(low=3, high=10),
    'min_samples_split': randint(low=2, high=10) 
}

print("\n--- 2단계: 탐색할 하이퍼파라미터 분포 정의 완료 ---")

In [ ]:
# --------------------
#  3단계: Random Search 실행 및 최적 조합 찾기
# --------------------
# estimator: 사용할 모델 (랜덤 포레스트)
# param_distributions: 정의된 탐색 공간 (확률 분포)
# n_iter=20: 무작위로 20가지 조합만 샘플링하여 시도
# cv=5: 5-Fold 교차 검증 사용
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    n_iter=20, # Grid Search와 달리 시도 횟수를 제한
    cv=5, 
    scoring='accuracy',
    random_state=42,
    n_jobs=-1 
)

# 학습 데이터에 Random Search 실행
random_search.fit(X_train, y_train)


In [ ]:
# --------------------
# 4 단계: 결과 확인
# --------------------
print("\n--- 4단계: 최적의 하이퍼파라미터 조합 및 성능 ---")
# 최적의 하이퍼파라미터 조합 (Best Parameters)
print("Best Parameters:", random_search.best_params_)

# 해당 조합으로 얻은 최고 교차 검증 점수 (Best Score)
print(f"Best CV Score (Accuracy): {random_search.best_score_:.4f}")

# 최종 모델
best_model = random_search.best_estimator_
final_test_score = best_model.score(X_test, y_test)
print(f"Final Test Score: {final_test_score:.4f}")